## Problem Set 6: Predictive Modeling

### Name: Gabriella Rivera

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

#### 1 Predict Airbnb Price 

1.

In [27]:
airbnb = pd.read_csv('airbnb-bangkok-listings.csv.bz2', sep=',', usecols=['bedrooms', 'price', 'accommodates', 'room_type'])
airbnb.head()

,room_type,accommodates,bedrooms,price
0,Entire home/apt,3,1.0,"$1,845.00"
1,Private room,2,1.0,"$1,275.00"
2,Private room,2,1.0,$800.00
3,Private room,2,1.0,$800.00
4,Private room,2,1.0,"$1,845.00"


In [28]:
airbnb['price'] = airbnb['price'].str.replace('$', '')  # Remove dollar sign
airbnb['price'] = airbnb['price'].str.replace(',', '')  # Remove commas

airbnb['price'] = pd.to_numeric(airbnb['price'])

In [29]:
clean_airbnb = airbnb.dropna(subset=['bedrooms', 'price', 'accommodates', 'room_type'])

In [30]:
clean_airbnb.isna().any()

room_type       False
accommodates    False
bedrooms        False
price           False
dtype: bool

2.

In [31]:
clean_airbnb['bedrooms'] = clean_airbnb['bedrooms'].astype(int)
clean_airbnb.loc[clean_airbnb['bedrooms'] >= 4, 'bedrooms'] = 4

C:\Users\Gabriella Rivera\AppData\Local\Temp\ipykernel_4632\2925388353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_airbnb['bedrooms'] = clean_airbnb['bedrooms'].astype(int)


In [32]:
m = smf.ols('np.log(price) ~ bedrooms + room_type + accommodates', data = clean_airbnb).fit()
m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(price)   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.237
Method:                 Least Squares   F-statistic:                     945.5
Date:                Thu, 16 May 2024   Prob (F-statistic):               0.00
Time:                        10:27:02   Log-Likelihood:                -16582.
No. Observations:               15200   AIC:                         3.318e+04
Df Residuals:                   15194   BIC:                         3.322e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     6.5849      0.015    449.007      0.000       6.556       6.614
room_type[T.Hotel room]       0.1047      0.028      3.799      0.000       0.051       0.159
room_type[T.Private room]    -0.0856      0.013     -6.745      0.000      -0.110      -0.061
room_type[T.Shared room]     -0.7983      0.030    -26.701      0.000      -0.857      -0.740
bedrooms                      0.4171      0.012     35.224      0.000       0.394       0.440
accommodates                  0.0394      0.004     11.136      0.000       0.032       0.046
==============================================================================
Omnibus:                     4221.820   Durbin-Watson:                   1.636
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22128.885
Skew:                           1.241   Prob(JB):                         0.00
Kurtosis:                       8.365   Cond. No.                         22.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

2.

In [33]:
phat = m.predict()
phat
# phat[:10]

array([7.12010689, 6.995152  , 6.995152  , ..., 7.6159347 , 7.08074826,
       7.12010689])

3.

In [34]:
rmse = np.sqrt(np.mean((phat - np.log(clean_airbnb.price))**2))
rmse

0.7203383032636651

4.

In [35]:
newDat = {"bedrooms": [2], "accommodates": [4], "room_type": ["Entire home/apt"]}
predicted_log_price = m.predict(newDat)
predicted_log_price

0    7.576576
dtype: float64

In [36]:
predicted_price = np.exp(predicted_log_price)
predicted_price                          

0    1951.934226
dtype: float64

5.

In [37]:
filtered_df = clean_airbnb[(clean_airbnb['bedrooms'] == 2) & (clean_airbnb['accommodates'] == 4)]
filtered_df

,room_type,accommodates,bedrooms,price
25,Entire home/apt,4,2,1174.0
30,Entire home/apt,4,2,2516.0
35,Entire home/apt,4,2,1747.0
60,Entire home/apt,4,2,2684.0
66,Private room,4,2,2000.0
...,...,...,...,...
16982,Entire home/apt,4,2,2000.0
16983,Entire home/apt,4,2,2800.0
17009,Entire home/apt,4,2,2179.0
17010,Entire home/apt,4,2,2179.0


In [38]:
average_log_price = np.mean(np.log(filtered_df['price']))
average_log_price

7.728450922007424

In [39]:
diff = average_log_price - predicted_log_price
diff

0    0.151875
dtype: float64

The average log price is 0.151875 greater than the predicted log price

#### Heart Attack: predictive modeling 

##### 2.1 Logit with sklearn

1.

In [40]:
heart = pd.read_csv('heart.csv.bz2', sep=',')
heart

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [41]:
heart.isna().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

2.

In [42]:
# Drop the 'output' column to create the design matrix X
X = heart.drop(columns=['output'])
X

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


3.

The design matrix should not contain output because output is the outcome, our outcome vector, our y value, our intercept. 

4.

In [43]:
# Generate dummy variables
X1 = pd.get_dummies(X, columns=["cp", "restecg"], drop_first=True, dtype=int)

X1.sample(5)

,age,sex,trtbps,chol,fbs,thalachh,exng,oldpeak,slp,caa,thall,cp_1,cp_2,cp_3,restecg_1,restecg_2
276,58,1,146,218,0,105,0,2.0,1,1,3,0,0,0,1,0
138,57,1,110,201,0,126,1,1.5,1,0,1,0,0,0,1,0
158,58,1,125,220,0,144,0,0.4,1,4,3,1,0,0,1,0
224,54,1,110,239,0,126,1,2.8,1,1,3,0,0,0,1,0
49,53,0,138,234,0,160,0,0.0,2,0,2,0,0,0,0,0


In [44]:
X1.shape

(303, 16)

5.

In [45]:
y = heart['output']

In [49]:
m = LogisticRegression(max_iter=1000)
_ = m.fit(X, y)

In [50]:
m.intercept_, m.coef_

(array([2.8191059]),
 array([[-0.00327444, -1.45322519,  0.81256887, -0.01846228, -0.00395978,
          0.00827912,  0.41304711,  0.02327379, -0.83815867, -0.53764958,
          0.50108642, -0.72796526, -0.84537372]]))

##### Predictive Modeling

1.

In [53]:
phat = m.predict_proba(X)[:,1]
phat[:10]

array([0.80455614, 0.66207326, 0.94538783, 0.90739092, 0.79779649,
       0.75629277, 0.79765398, 0.84329378, 0.77663191, 0.90751487])

2.

In [54]:
yhat = m.predict(X)
yhat[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

3.

In [56]:
yhat = phat > .5
yhat[:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

4.